In [ ]:
import os
from bb import *
import pandas as pd
import networkx as nx
import plotly.express as px
import matplotlib.pyplot as plt

### Plot NMR Graph

In [ ]:
def plot_graph(G):
    pos = nx.spring_layout(G)    
    node_color = ['y' if ':' in node else 'r' for node in G.nodes()]
    node_size = [800 if ':' in node else 300 for node in G.nodes()]
    nx.draw_networkx_nodes(G, pos, 
        node_size=node_size, 
        node_shape='o', 
        node_color=node_color)
    nx.draw_networkx_labels(G, pos, font_color='k', font_size=8)
    nx.draw_networkx_edges(G, pos)    
    plt.show()

In [ ]:
# fnmr = '../DATA_TEST/testE.nmr'
fnmr = '../DATA_EPSD_00_DMAX_60/1cza.nmr'
nmr = NMR(fnmr)
E, S = nmr.ordering_data
G = nmr.ordering_graph
plot_graph(G)

In [ ]:
order_cost([3,2,1],E, S)

### READ LOG FILES

In [ ]:
WDIR = ['../DATA_EPSD_00_DMAX_50', '../DATA_EPSD_00_DMAX_60']
df = []
for wdir in WDIR:
    dmax = int(wdir.split('_')[-1])
    for flog in sorted(os.listdir(wdir)):
        if not flog.endswith('.log'):
            continue
        pid = flog.replace('.log','')
        flog = os.path.join(wdir, flog)
        # check if the problem was pickled
        pkl = os.path.exists(flog.replace('.log','.pkl'))
        df_log = {'pid': pid, 'dmax':dmax, 'flog': flog, 'pkl': pkl}
        with open(flog, 'r') as fid:
            for row in fid:
                row = row.replace(':','').replace('\n','').split(' ')
                field = row[1]
                if 'fnmr' in field:
                    value = row[-1]
                elif 'timeoutBB' in field:
                    value = bool(row[-1] == 'True')
                elif 'time' in field:
                    value = float(row[-1])
                else:
                    value = int(row[-1])                    
                df_log[field] = value
        if ('costSBBU' in df_log) and (df_log['costSBBU'] == df_log['costRELAX']):
            df_log['costBB'] = df_log['costSBBU']
            df_log['timeBB'] = df_log['timeSBBU']
        df.append(df_log)
df = pd.DataFrame(df)
df['gapRL'] = (df['costSBBU'] - df['costRELAX']) / df['costRELAX']
df['gapBB'] = (df['costSBBU'] - df['costBB']) / df['costBB']
fname = '../results.xlsx'
print('Save', fname)
df.to_excel(fname,index=False)
df.head()

In [ ]:
df.groupby(by=['pid','dmax']).mean()

In [ ]:
df50 = df[df['dmax']==50]
df60 = df[df['dmax']==60]